### Main goals:
Clarify and reflect on the definition of the term "fake news", which may vary among databases, sometimes non-binary.\
Research, where the data comes from and inspect the data: what are the labels, sources, and authors?\
Is there a person, source or topic which is over- or under-represented?\
Study the literature on how others approach this task. Check the related literature and select your model architecture of choice: LSTM, ...\
Develop a classification model to predict fake news from the text. How do you judge the quality of your results, i.e. which metrics do you consider?
### Optional:
Inspect the falsely classified ones. What can you learn from them?\
Investigate edge cases that you found in your data inspection with respect to how the model learned to identify these.\
Experiment with how you could mitigate if edge cases are covered poorly.


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments,Trainer, pipeline
from datasets import load_dataset
import numpy as np
import evaluate
from sklearn.metrics import classification_report
from bertviz import model_view,head_view
import shap

In [2]:
# model_name = "google-bert/bert-base-uncased"
model_name = 'bert-base-uncased-fine-tuned'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [5]:
data = load_dataset('GonzaloA/fake_news')

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
data = data.remove_columns(['Unnamed: 0','title'])

In [7]:
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = tokenized_data["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_data["validation"].shuffle(seed=42).select(range(100))
small_test_dataset = tokenized_data['test'].select(range(100))

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,output_attentions = True)

In [ ]:
# pipe = pipeline('text-classification',model=model_name, return_all_scores=True)

In [ ]:
# explainer = shap.Explainer(pipe)

In [ ]:
# shap_values = explainer([data['train']['text'][3]])

In [36]:
inputs = tokenizer.encode(small_test_dataset[9]['text'], return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [46]:
# head_view(attention, tokens)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [10]:
pipe = pipeline('text-classification',model=model_name, top_k=None)

In [11]:
explainer = shap.Explainer(pipe)

In [44]:
shap_values = explainer(['The price of fried chicken is outrageous now.'])

  0%|          | 0/110 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:13, 13.88s/it]               


In [45]:
shap.plots.text(shap_values)

In [ ]:
test_labels = small_test_dataset['label']
small_test_dataset = small_test_dataset.remove_columns(['label','token_type_ids'])

In [ ]:
predictions = trainer.predict(small_test_dataset)

In [ ]:
predicted_labels = predictions.predictions.argmax(axis=1)

In [ ]:
predicted_labels

In [ ]:
print(classification_report(test_labels, predicted_labels))

In [ ]:
tokenizer.save_pretrained('model2')